This will set the current working directory to the place where the .ipynb file resides

In [ ]:
import os
os.chdir(os.getcwd())

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import xlsxwriter
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Defining the mean absolute deviation function without importing Statsmodels library

Alternatively I could use [Pandas built-in mad function](https://stackoverflow.com/a/44629090/4999991)

In [ ]:
# mean absolute deviation function
def mad(data, axis=None):
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

Defining the linear function:

In [ ]:
def func(x, a, b):
    return a * x + b

Defining some general information about the experiments:

In [ ]:
speed=[50,100,150,200,300,400,600,800,1000,1200,1400,1600,1800]
#weights=['44','80','128','188','236']
#weights=[44,80,128,188,236]
headers=('static_strain(mm)','static_force','min_mean','min_del','min_del_mad','max_mean','max_del','max_del_mad')
#df= pd.DataFrame(columns=headers,index=map(float,weights))
# df= pd.DataFrame(columns=headers,index=weights)

headers2=['static_force_a','static_force_b','static_strain_a','static_strain_b','kinetic_max_mean_a','kinetic_max_mean_b','kinetic_min_mean_a','kinetic_min_mean_b','kinetic_del_avg','kintetic_del_mad','mad/avg']
df2= pd.DataFrame(columns=headers2,index=speed)

areas=[900,1700,2500]

surcons=('bs perpendicular','ss parallel parallel','ss parallel perpendicular','ss perpendicular perpendicular')
surcon='ss parallel parallel'

This will read all the excel files and export Analysis files per speed and Area

In [ ]:
# writer2 = pd.ExcelWriter(surcon+'\\analysis.xlsx', engine='xlsxwriter')
for a in areas:
    #     writer = pd.ExcelWriter(surcon+'\\'+str(a)+'mm\\analysis.xlsx', engine='xlsxwriter')
    for s in speed:
        file = surcon + '\\' + str(a) + 'mm\\' + str(a) + 'mm_' + str(s) + 'mm_min.xls'
        xls = pd.ExcelFile(file)
        #print(xls.sheet_names[3:])
        weights = [int(w[:-1]) for w in xls.sheet_names[3:]]
        #print(a,',',s,',',weights)
        df = pd.DataFrame(columns=headers, index=weights)
        for j in weights:
            dfs = pd.read_excel(file, sheet_name=str(j) + 'g')

            #clean up columns
            dfs.columns = ['time', 'strain', 'force', 'work']
            dfs = dfs.drop(dfs.index[[0, 1]])
            max = dfs['force'].max()
            #print(max)
            #print(dfs[dfs['force']==dfs['force'].max()])
            #print(dfs[dfs['force']==max].strain.value)
            #print(j,',',dfs[dfs['force']==max].strain.values[0],',',max,',', end='')
            #df.loc[df.GPA == df.GPA.max(), 'ID'].values[0]
            df[headers[0]][j] = dfs[dfs['force'] == max].strain.values[0]
            df[headers[1]][j] = max

            n = 9  # the bigger the more comparison width
            dfs['min'] = dfs.iloc[argrelextrema(
                dfs.force.values, np.less_equal, order=n)[0], 2]
            dfs['max'] = dfs.iloc[argrelextrema(
                dfs.force.values, np.greater_equal, order=n)[0], 2]

            #removing the NaNs
            dfs2 = dfs.dropna(subset=['min'])
            dfs3 = dfs.dropna(subset=['max'])

            #calculating the delta
            pd.options.mode.chained_assignment = None  # default='warn'
            dfs2['del'] = dfs2.strain - dfs2.strain.shift(1)
            dfs3['del'] = dfs3.strain - dfs3.strain.shift(1)

            #removing the too close points
            dfs2 = dfs2[dfs2['del'] > 0.5 * dfs2['del'].mean()]
            dfs3 = dfs3[dfs3['del'] > 0.5 * dfs3['del'].mean()]

            dfs2 = dfs2[dfs2['del'] < 1.9 * dfs2['del'].mean()]
            dfs3 = dfs3[dfs3['del'] < 1.9 * dfs3['del'].mean()]

            #removing NaNs
            dfs2 = dfs2.dropna(subset=['del'])
            dfs3 = dfs3.dropna(subset=['del'])

            #calculating mean frequency and mean absolute deviation
            #print(dfs2['min'].mean(),',',dfs2['del'].mean(),',',mad(dfs2['del'].values), end='')
            df[headers[2]][j] = dfs2['min'].mean()
            df[headers[3]][j] = dfs2['del'].mean()
            df[headers[4]][j] = mad(dfs2['del'].values)
            #print(dfs3['max'].mean(),',',dfs3['del'].mean(),',',mad(dfs3['del'].values))
            df[headers[5]][j] = dfs3['max'].mean()
            df[headers[6]][j] = dfs3['del'].mean()
            df[headers[7]][j] = mad(dfs3['del'].values)

        #print(df)
        df.to_excel(writer, sheet_name=str(s))
        #print(df[headers[0]].values.ravel())
        #print(df[headers[0]].values)
        #print(df[headers[0]].values.astype(float))
        #print(df[headers[0]])
        popt, pcov = curve_fit(func, weights,
                               df[headers[0]].values.astype(float))
        #print(popt)
        df2[headers2[2]][s] = popt[0]
        df2[headers2[3]][s] = popt[1]

        popt, pcov = curve_fit(func, weights,
                               df[headers[1]].values.astype(float))
        df2[headers2[0]][s] = popt[0] * 1000 / 9.81
        df2[headers2[1]][s] = popt[1]

        popt, pcov = curve_fit(func, weights,
                               df[headers[5]].values.astype(float))
        df2[headers2[4]][s] = popt[0] * 1000 / 9.81
        df2[headers2[5]][s] = popt[1]

        popt, pcov = curve_fit(func, weights,
                               df[headers[2]].values.astype(float))
        df2[headers2[6]][s] = popt[0] * 1000 / 9.81
        df2[headers2[7]][s] = popt[1]

        df2[headers2[8]][s] = (
            df[headers[3]].mean() + df[headers[6]].mean()) / 2
        df2[headers2[9]][s] = (
            df[headers[4]].mean() + df[headers[7]].mean()) / 2
    #     df2[headers2[10]][s]=

    writer.save()
    df2[headers2[10]] = df2[headers2[9]] / df2[headers2[8]]
    df2.to_excel(writer2, sheet_name=str(a))

# writer2.save()

In [ ]:
#%matplotlib inline
%matplotlib qt
plt.plot(dfs['strain'], dfs['force'],label='data')
plt.scatter(dfs2['strain'], dfs2['min'],label='min')
plt.scatter(dfs3['strain'], dfs3['max'],label='max')
plt.show()

In [ ]:
#%matplotlib inline
%matplotlib qt
plt.plot(df2.index, df2[headers2[0]],label='data')
plt.show()

In [ ]:
#%matplotlib inline
%matplotlib qt
file2=surcon+'\\analysis.xlsx'
for a in areas:
    df3 = pd.read_excel(file2, sheet_name=str(a))
    plt.plot(df3.index, df3[headers2[4]],label=str(a))
    #plt.scatter(df3[headers2[10]], df3[headers2[10]],label=str(a))

plt.show()

Calculating the nonlinear models
$$F_f=aF_N^n+cF_N$$
where is $n$ is either $1/2$ or $2/3$

In [ ]:
def func2(x, a, c):
    return a * x**(2/3.0) +c*x

In [ ]:
#%matplotlib inline
# %matplotlib qt
writer3 = pd.ExcelWriter(surcon + '\\analysis_1.xlsx', engine='xlsxwriter')
file4 = surcon + '\\analysis.xlsx'
xls3 = pd.ExcelFile(file4)
areas2 = [int(a) for a in xls3.sheet_names]
# print(areas2)
for a in areas2:
    df5 = pd.read_excel(file4, index_col=None, sheet_name=str(a))
    df5 = df5.reset_index().dropna()
    df5['index'] = df5['index'].astype(int)
    df5 = df5.set_index('index')

    df5['new_a'] = np.nan
    df5['new_c'] = np.nan

    file3 = surcon + '\\' + str(a) + 'mm\\analysis.xlsx'
    xls2 = pd.ExcelFile(file3)
    speeds2 = [int(s) for s in xls2.sheet_names]
    for s in speeds2:
        df4 = pd.read_excel(file3, sheet_name=str(s))
        df4 = df4.reset_index().dropna()
        df4['index'] = df4['index'].astype(int)
        df4 = df4.set_index('index')
        popt, pcov = curve_fit(
            func2,
            df4.index,
            df4[headers[1]].values.astype(float),
            maxfev=1000,
            p0=[df5[headers2[1]][s], df5[headers2[0]][s]])
        #print(popt)
        df5['new_a'][s] = popt[0]
        df5['new_c'][s] = popt[1]
    df5[['new_a', 'new_c']].to_excel(writer3, sheet_name=str(a))
    #print(df5[['new_a','new_c']])
writer3.save()
# plt.plot(df5.index, df5['new_a'],label='a')
# plt.plot(df5.index, df5['new_c'],label='c')
# plt.show()

comparing linear and nonleaner models for friction 

In [ ]:
%matplotlib qt

areas2 = [900, 1700, 2500]

file_lin = surcon + '\\analysis_lin.xlsx'
xls_lin = pd.ExcelFile(file_lin)

file_05 = surcon + '\\analysis_05.xlsx'
xls_05 = pd.ExcelFile(file_05)

file_23 = surcon + '\\analysis_23.xlsx'
xls_23 = pd.ExcelFile(file_23)

error_lin = 0.0
error_05 = 0.0
error_23 = 0.0

for a in areas2:
    file_a = surcon + '\\' + str(a) + 'mm\\analysis.xlsx'
    xls_a = pd.ExcelFile(file_a)
    speeds_a = [int(s) for s in xls_a.sheet_names]

    df_lin = pd.read_excel(file_lin, sheet_name=str(a))

    df_05 = pd.read_excel(file_05, sheet_name=str(a))
    df_05 = df_05.dropna()
    df_05['index'] = df_05['index'].astype(int)
    df_05 = df_05.set_index('index')

    df_23 = pd.read_excel(file_23, sheet_name=str(a))
    df_23 = df_23.dropna()
    df_23['index'] = df_23['index'].astype(int)
    df_23 = df_23.set_index('index')

    for s in speeds_a:
        df_s = pd.read_excel(file_a, sheet_name=str(s))
        #print(a,',',s)
        error_lin += np.sum(
            np.absolute(df_s.static_force.values - (
                df_lin['static_force_a'][s] * df_s.index.values * 9.81 / 1000 +
                df_lin['static_force_b'][s])))
        error_05 += np.sum(
            np.absolute(df_s.static_force.values -
                        (df_05['new_a'][s] * df_s.index.values**0.5 +
                         df_05['new_c'][s] * df_s.index.values)))
        error_23 += np.sum(
            np.absolute(df_s.static_force.values -
                        (df_23['new_a'][s] * df_s.index.values**
                         (2.0 / 3.0) + df_23['new_c'][s] * df_s.index.values)))
        #print(np.absolute(df_05['new_c'][s]))
        #print(df_lin['static_force_a'][s])

print('lin: ', error_lin)
print('05: ', error_05)
print('23: ', error_23)
# plt.scatter(df_s.index, df_s.static_force.values,label='org')
# plt.plot(df_s.index, (df_lin['static_force_a'][s]*df_s.index.values*9.81/1000-df_lin['static_force_b'][s]),label='lin')
# plt.plot(df_s.index, (df_05['new_a'][s]*df_s.index.values**0.5+df_05['new_c'][s]*df_s.index.values),label='0.5')
# plt.plot(df_s.index, (df_23['new_a'][s]*df_s.index.values**(2.0/3.0)+df_23['new_c'][s]*df_s.index.values),label='2/3')
# plt.legend()
# plt.show()

lin:  4.570348107914248
05:  4.767694822076519
23:  4.8696002018532685
